In [1]:
!pip install nltk

In [6]:
import nltk
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jminw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# 뉴스 다운로드 및 전처리
def get_news():
    #20newsgroup 다운로드
    dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers','quotes'))
    documents = dataset.data
    print(documents)
    news_df = pd.DataFrame({'document':documents})
    
    #전처리
    news_df['clean_doc'] = news_df['document'].str.replace('[^a-zA-Z]', " ") # 특수문자제거
    news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : ' '.join([w for w in x.split() if len(w)>3]))
    news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : x.lower()) # 전체 단어에 대한 소문자 변환
    
    tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
    
    stop_words = stopwords.words('english') #NLTK 불용어조회
    return tokenized_doc.apply(lambda x: ' '.join([item for item in x if item not in stop_words]))

C:\Data\envs\py37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
tokenized_docs = get_news()

In [12]:
tokenized_docs

0        well sure story seem biased disagree statement...
1        yeah expect people read actually accept hard a...
2        although realize principle strongest points wo...
3        notwithstanding legitimate fuss proposal much ...
4        well change scoring playoff pool unfortunately...
                               ...                        
11309    danny rubenstein israeli journalist speaking t...
11310                                                     
11311    agree home runs clemens always memorable kinda...
11312    used deskjet orange micros grappler system upd...
11313    argument murphy scared hell came last year han...
Name: clean_doc, Length: 11314, dtype: object

In [13]:
type(tokenized_docs)

pandas.core.series.Series

In [14]:
# 공백으로 토큰 분리
def my_tokenizer(text):
    return text.split()

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer(tokenizer = my_tokenizer)
tfidf = tfidf_vect.fit_transform(tokenized_docs)

#모델 선언
lda = LatentDirichletAllocation(n_components=20)
lda_output = lda.fit_transform(tfidf)

In [17]:
!pip install pyLDAvis

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97717 sha256=f63a767dab7d27e1ad9dc8b3c8bea09b38b6b0e64153d18d0121991c1f703717
  Stored in directory: c:\users\jminw\appdata\local\pip\cache\wheels\3b\fb\41\e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491062 sha256=95d564828d5fba6859aab1a336ae68677f2024c4a25c99796d1f8b51c4605c68
  Stored in directory: c:\users\jminw\appdata\local\pip\cache\wheels\56\b0\fe\4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32047 sha256=74745755ad30e2f652f5a768bbd5b2db870fa0175a4e565a971b594b78de96cc
  Stored in directory: c:\users\jminw\appdata\local\pip\cache\wheels\3c\33\97\805b282e129f60bb4e87cea622338f30b65f21eaf65219971f
Successfully built pyLDAvis future funcy


In [19]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)

C:\Data\envs\py37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
